# Customer Churn Analysis (Banking Dataset)

## Executive Summary
In this analysis I explore customer churn (`Exited`) to identify the main patterns and high-risk segments.  
Goal: produce actionable insights and practical recommendations.

After the Analysis, I'm gonna add the costruction of a Machine Learning Model that focuses on predict Churn or Retention value of a new customer set.

**Key outputs**
- Overall churn rate
- Top churn drivers (by segment comparison)
- 2–3 high-risk segments
- 3 actionable recommendations + how to measure impact


### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Reading the Dataset

In [2]:
df = pd.read_csv('../data/churn.csv')

## 1) Defining context

### Business question
Which customer segments are more likely to churn, and what patterns can explain churn behavior?

### Target definition
`Exited`  
- 1 = customer churned  
- 0 = customer retained

### Unit of analysis
1 row = 1 customer

### Scope
This project focuses on:
- data checks & data quality
- exploratory analysis (univariate / bivariate / multivariate)
- segmentation and business recommendations  
- prediction of the future customers churn creating a machine learning model.


## 2) Dataset sanity check

### What I check
- Dataset size (rows/columns)
- Column types and basic structure
- Duplicates (row-level and customer ID if available)
- Target distribution (churn rate baseline)

### Notes
- Identifier-like columns (e.g., `RowNumber`, `CustomerId`) are used only for checks and will not be treated as analytical features.


In [7]:
# --- 1) Quick preview: capisci subito com'è fatto ---
print("Shape (rows, cols):", df.shape)
display(df.head(3))

# --- 2) Column overview: tipi, null, memoria (molto utile) ---
display(df.info())

# --- 3) Check duplicate rows ---
dup_rows = df.duplicated().sum()
print("Duplicate rows:", dup_rows)

# --- 4) Check duplicate customer IDs (se presente) ---
if "CustomerId" in df.columns:
    dup_ids = df["CustomerId"].duplicated().sum()
    print("Duplicate CustomerId:", dup_ids)

# --- 5) Target sanity: Exited deve essere 0/1 e non vuoto ---
assert "Exited" in df.columns, "Target column 'Exited' not found!"

print("\nExited value counts (absolute):")
print(df["Exited"].value_counts(dropna=False))

print("\nExited value counts (percentage):")
print((df["Exited"].value_counts(normalize=True, dropna=False) * 100).round(2))

# controllo: ci aspettiamo solo 0 e 1
unique_target = set(df["Exited"].dropna().unique())
print("\nUnique values in Exited:", unique_target)

# Se vuoi essere strict:
# assert unique_target.issubset({0,1}), f"Unexpected target values: {unique_target}"

# --- 6) Identify ID-like columns (da escludere dall'analisi) ---
id_like = [c for c in ["RowNumber", "CustomerId", "Surname"] if c in df.columns]
print("\nID-like columns detected (do not use as features):", id_like)

# --- 7) Basic range checks (solo se la colonna esiste) ---
def quick_range(col):
    return {"min": df[col].min(), "max": df[col].max()}

for col in ["Age", "Tenure", "CreditScore", "NumOfProducts", "Balance", "EstimatedSalary"]:
    if col in df.columns:
        print(col, quick_range(col))

# --- 8) Spot-check suspicious cases (opzionale ma utile) ---
# Esempi: età troppo alta/bassa, tenure fuori range, ecc.
if "Age" in df.columns:
    print("\nCustomers with Age < 18:", (df["Age"] < 18).sum())
    print("Customers with Age > 100:", (df["Age"] > 100).sum())

if "Tenure" in df.columns:
    print("Tenure < 0:", (df["Tenure"] < 0).sum())
    print("Tenure > 10:", (df["Tenure"] > 10).sum())


Shape (rows, cols): (10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  str    
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  str    
 5   Gender           10000 non-null  str    
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), str(3)
memory usage: 1.1 MB


None

Duplicate rows: 0
Duplicate CustomerId: 0

Exited value counts (absolute):
Exited
0    7963
1    2037
Name: count, dtype: int64

Exited value counts (percentage):
Exited
0    79.63
1    20.37
Name: proportion, dtype: float64

Unique values in Exited: {np.int64(0), np.int64(1)}

ID-like columns detected (do not use as features): ['RowNumber', 'CustomerId', 'Surname']
Age {'min': np.int64(18), 'max': np.int64(92)}
Tenure {'min': np.int64(0), 'max': np.int64(10)}
CreditScore {'min': np.int64(350), 'max': np.int64(850)}
NumOfProducts {'min': np.int64(1), 'max': np.int64(4)}
Balance {'min': np.float64(0.0), 'max': np.float64(250898.09)}
EstimatedSalary {'min': np.float64(11.58), 'max': np.float64(199992.48)}

Customers with Age < 18: 0
Customers with Age > 100: 0
Tenure < 0: 0
Tenure > 10: 0


The dataset contains 10,000 customers and 14 columns, suitable for exploratory analysis and segmentation.

No missing values were detected (all columns are 100% non-null), so analysis can proceed without imputation steps.

No duplicate rows or duplicate CustomerId values were found, supporting the assumption that each row represents a unique customer, reducing the risk of double-counting in churn rate estimates.

Overall churn rate is 20.37% (2037/10000). Target values are clean and binary (0/1) with a moderate class imbalance.

RowNumber, CustomerId, and Surname are identifier-like fields and will not be used as analytical features (useful only for integrity checks).

All numeric ranges appear plausible. Balance and EstimatedSalary show wide ranges and may be skewed, so distributions and outliers will be inspected during EDA.


## 3) Data quality

### Missing values
- Check missing values per column (%)
- Decide whether missing values require imputation or exclusion

### Valid ranges & constraints (quick checks)
- Age: reasonable range (e.g., 18+)
- Tenure: expected bounds (e.g., 0–10)
- CreditScore: typical bounds (e.g., 300–850)
- NumOfProducts: expected bounds (e.g., 1–4)
- Balance / EstimatedSalary: non-negative values

### Outliers & skew
Identify variables with heavy tails or extreme values (often Balance/Salary).


## 4) Target analysis (baseline)

### Overall churn rate
Compute the baseline churn rate and comment on class balance:
- Is churn rare (<10%), moderate (10–30%), or high (>30%)?
- Why class balance matters (interpretation of segments and later modeling).


## 5) Univariate analysis

### Numerical variables
For each key numerical variable (e.g., Age, Tenure, CreditScore, Balance, EstimatedSalary):
- distribution (histogram)
- spread/outliers (boxplot)
- short note: “what stands out?”

### Categorical variables
For each categorical/binary variable (e.g., Geography, Gender, HasCrCard, IsActiveMember, NumOfProducts):
- frequency table (%)
- short note: “dominant groups / imbalance?”


## 6) Bivariate analysis with target (core section)

### Goal
Quantify churn differences across groups:
- churn rate by category
- churn rate across binned numerical variables (e.g., Age groups)

### What I produce
For each key feature:
- churn rate per group
- group size (important: avoid misleading small groups)
- quick takeaway: “higher/lower churn than baseline?”

### Examples of comparisons
- Churn by Geography
- Churn by IsActiveMember
- Churn by NumOfProducts
- Churn by AgeGroup (binned)
- Churn by Balance presence (Balance = 0 vs > 0)


## 7) Multivariate analysis (segment discovery)

### Goal
Find practical high-risk segments using combinations of variables.

### Approach
Create 2–3 simple cross-segment views (pivot tables), such as:
- Geography × IsActiveMember → churn rate
- AgeGroup × IsActiveMember → churn rate
- NumOfProducts × HasBalance → churn rate

### Output
A short list of high-risk segments with:
- churn rate
- segment size
- why this segment might churn (hypothesis)


## 8) Key findings (final)

Write 5–7 findings max. Each finding must include:
- the segment/variable
- the churn rate (or difference vs baseline)
- a one-line interpretation

**Template**
1) **Finding:** …  
   **Evidence:** churn = __% vs baseline __% (n=__)  
   **Interpretation:** …

2) **Finding:** …  
   **Evidence:** …  
   **Interpretation:** …


## 9) Recommendations (business actions)

Provide 3 practical actions tied to findings.

For each recommendation include:
- **Who** (which segment)
- **What** (action)
- **Why** (based on findings)
- **How to measure** (metrics before/after)

**Template**
1) **Segment:** …  
   **Action:** …  
   **Rationale:** …  
   **Measure:** churn rate in this segment, retention at 30/60/90 days, etc.


## 10) Limitations & next steps

### Limitations
- This is an observational dataset: correlation ≠ causation
- Some variables may be proxies (e.g., geography may correlate with pricing/product differences)
- Snapshot data: churn behavior may change over time

### Next steps
- Optional: add a simple baseline model for validation (logistic regression) with interpretability
- Build a small dashboard (Power BI) to monitor churn by segment over time (if data available)


### Splitting the dataset into indipendet variables(X) and dipendent variable (y)

In [17]:
X = df.drop("Exited", axis=1)
y = df["Exited"]

### Encoding Categorical Data: Geography and Gender


In [18]:
categorical_cols = ["Geography", "Gender"]
numerical_cols = X.drop(columns=categorical_cols).columns

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", sparse_output=False), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)
X_clean = preprocessor.fit_transform(X)

In [30]:
# nomi colonne one-hot
ohe_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_cols)

# colonne numeriche
all_feature_names = list(ohe_feature_names) + list(numerical_cols)

In [32]:
X_df = pd.DataFrame(
    X_clean,columns=all_feature_names
)

In [33]:
X_df.head()
X_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Geography_Germany  10000 non-null  object
 1   Geography_Spain    10000 non-null  object
 2   Gender_Male        10000 non-null  object
 3   RowNumber          10000 non-null  object
 4   CustomerId         10000 non-null  object
 5   Surname            10000 non-null  str   
 6   CreditScore        10000 non-null  object
 7   Age                10000 non-null  object
 8   Tenure             10000 non-null  object
 9   Balance            10000 non-null  object
 10  NumOfProducts      10000 non-null  object
 11  HasCrCard          10000 non-null  object
 12  IsActiveMember     10000 non-null  object
 13  EstimatedSalary    10000 non-null  object
dtypes: object(13), str(1)
memory usage: 1.1+ MB
